### Libarary load

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',  None)

### Dataset load

In [3]:
# # patient
patient = pd.read_csv('../Data/mimic/core/patients.csv')
admissions = pd.read_csv('../Data/mimic/core/admissions.csv')

# medication
inputevents = pd.read_csv('../Data/mimic/icu/inputevents.csv')

# output

outputevents = pd.read_csv('../Data/mimic/icu/outputevents.csv')

# fluids
ingredientevents = pd.read_csv('../Data/mimic/icu/ingredientevents.csv')

#procedureevent
procedureevents = pd.read_csv('../Data/mimic/icu/procedureevents.csv')

# item
d_items = pd.read_csv('../Data/mimic/icu/d_items.csv')
d_labitems = pd.read_csv('../Data/mimic/hosp/d_labitems.csv')

# chartevent
chartevent = pd.read_csv('../Data/mimic/icu/chartevents.csv')

# labevent
labevents = pd.read_csv('../Data/mimic/hosp/labevents.csv')

# microbio event
micro = pd.read_csv('../Data/mimic/hosp/microbiologyevents.csv')

# icu stay
stay = pd.read_csv('../Data/mimic/icu/icustays.csv')

# additional information
inf = pd.read_csv('../Data/mimic/icu/icustays_preprocessed.csv')
suspected_infection = pd.read_csv('../Data/mimic/hosp/suspected_infection.csv')

# for judgment presence of heart failure 
d_icd = pd.read_csv('../Data/mimic/hosp/d_icd_diagnoses.csv')
diagnosses = pd.read_csv('../Data/mimic/hosp/diagnoses_icd.csv')

C:\Users\DAHS\AppData\Local\Temp\ipykernel_38596\1288757720.py:29: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  micro = pd.read_csv('../Data/mimic/hosp/microbiologyevents.csv')
C:\Users\DAHS\AppData\Local\Temp\ipykernel_38596\1288757720.py:32: DtypeWarning: Columns (3,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  stay = pd.read_csv('../Data/mimic/icu/icustays.csv')


In [4]:
# suspected infections

si_icd9_ary = suspected_infection[suspected_infection['icd_version']==9]['icd_code'].values
si_icd9 = np.array(si_icd9_ary[0].split(', '))

si_icd10_ary = suspected_infection[suspected_infection['icd_version']==10]['icd_code'].values
si_icd10 = np.array(si_icd10_ary[0].split(', '))

suspected_infection_icd_code_9_10 =np.hstack([si_icd9, si_icd10])

In [5]:
suspected_infection_df = pd.DataFrame()

for icd in suspected_infection_icd_code_9_10:

    sample = diagnosses[diagnosses['icd_code'].str.startswith(icd)][['subject_id', 'hadm_id']]
    suspected_infection_df = pd.concat([suspected_infection_df, sample])

In [6]:
heartfailure_before_df = pd.DataFrame()

for icd in ['428', 'I50']:

    sample = diagnosses[diagnosses['icd_code'].str.startswith(icd)][['subject_id', 'hadm_id']]
    heartfailure_before_df = pd.concat([heartfailure_before_df, sample])

In [7]:
spinal_injury_before_df = pd.DataFrame()

for icd in ['G50', '805', '806']:

    sample = diagnosses[diagnosses['icd_code'].str.startswith(icd)][['subject_id', 'hadm_id']]
    spinal_injury_before_df = pd.concat([spinal_injury_before_df, sample])

In [11]:
# target patient selection

def age_constraint(df):
    p = df.copy()
    
    result = p[ (p['age_at_intime']>17) | (p['age_at_intime']<100) ]
    selected_p_idx = result.index
    # static_value = result.loc[selected_p_idx,['stay_id','age_at_intime']]
    
    return selected_p_idx


def demo_inf(df):
    result = df.copy()
    
    selected_p_idx = age_constraint(df)
    static_value = result.loc[selected_p_idx,['subject_id','stay_id','intime', 'age_at_intime','race', 'gender']]
    
    return static_value

# patient_inf = demo_inf(inf)

In [26]:
len(patient.subject_id.unique())

299712

In [47]:
procedureevents.originalrate.value_counts()

originalrate
0    357589
1    338503
Name: count, dtype: int64

In [9]:
using_itm = d_items[['itemid','label']]
pe = procedureevents.rename(columns = {'value':'amount', 'originalrate': 'rate'})

def pharma(information, item,  df1, df2, df3):
    
    target_stay_id = information.stay_id.unique()
    
    sample1 = df1[['subject_id', 'hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime', 'amount', 'rate']].copy()
    sample2 = df2[['subject_id', 'hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime', 'amount', 'rate']].copy()
    sample3 = df3[['subject_id','hadm_id', 'stay_id', 'itemid', 'starttime', 'endtime', 'amount', 'rate']].copy()
    sample3['rate'] = sample3['rate'].replace(0, 1)
    
    result = pd.concat([sample1, sample2], axis = 0, ignore_index=True)
    result = pd.concat([result, sample3], axis = 0, ignore_index=True)

    condition = result.stay_id.isin(target_stay_id)
    total = result[condition]
    total.sort_values(by='stay_id', inplace = True)

    pharma_df = pd.merge(total, item, how = 'left', on = 'itemid')
    
    return pharma_df

pm = pharma(patient_inf, using_itm,  ingredientevents.fillna(0), inputevents.fillna(0), pe.fillna(0)) # all values are not missing, only missing at rate and it filled with value 0 it means they not have rate, since pe's "original_rate" is not drug rate, it replaced with 0

In [10]:
medications_item_id=[
    222318, 225975, 225152, 225157 ,221261, 222062, 221555, 229233, 221468, 221342, 221653, 221986, 221662, 229069,
    222056, 222051, 222037, 221429, 225153, 225974, 221906, 229630, 229632, 221749, 221289, 229617, 222315, 225166,
    225840, 225875, 225883, 225876, 225893, 229064, 225850, 225855, 225859, 225860, 225865, 225798, 225837, 225838,
    225842, 225843, 225844, 225845, 225847, 225848, 225851, 225857, 225859 ,225862, 225863, 225866, 225868, 225869, 
    225871, 225873, 225875, 225876, 225877 ,225879, 225881, 225882, 225883, 225884, 225885, 225886, 225888, 225889, 
    225890, 225892, 225893, 225895, 225896, 225897, 225898, 225899, 225900, 225902, 225903, 225905, 227691, 228003, 
    229059, 229061, 229064, 229587]

Fluids_item_id=[
    220490, 226222, 226224, 226509, 227080, 220949  
]

procedure_item_id=[
    225792, 225794, 225402, 223253, 225459, 225401, 225437
]

pmpe_item_id = medications_item_id + Fluids_item_id + procedure_item_id

condition = pm.itemid.isin(pmpe_item_id)
selected_pmpe= pm[condition]


pmpe_inf_start = selected_pmpe[['subject_id','stay_id','itemid', 'starttime', 'amount', 'rate', 'label']].copy()
pmpe_inf_end = selected_pmpe[['subject_id','stay_id','itemid', 'endtime', 'amount', 'rate','label']].copy()

pmpe_inf_start.rename(columns = {'starttime':'charttime'}, inplace = True)
pmpe_inf_end.rename(columns = {'endtime':'charttime'}, inplace = True)

pmpe = pd.concat([pmpe_inf_start, pmpe_inf_end], axis=0, ignore_index=True)
pmpe['charttime']=pd.to_datetime(pmpe['charttime'])
pmpe.sort_values(by=['stay_id', 'charttime'], inplace = True)

In [11]:
# signal: chartevents table
selected_chartevt = chartevent[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value']].copy()

# lab test: labevents table
selected_labevt = labevents[['subject_id', 'charttime', 'itemid', 'valuenum']].copy()

# micro event
selected_mi = micro[(micro['spec_itemid']==70011)| (micro['spec_itemid']==70012)][['subject_id', 'charttime', 'spec_itemid', 'test_seq']].copy()

# output event
selected_oe = outputevents[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value']].copy()

# item matching

def item_match(d_labitems, d_items, chart_sub, lab_sub, mi_sub, oe_sub):

    using_hsp_itm = d_labitems[['itemid', 'label']].copy()
    variable_name = d_items[['itemid', 'abbreviation']].copy()
    
    mi_sub = mi_sub.rename(columns = {'spec_itemid': 'itemid', 'test_seq':'value'})
    
    lab_sub = lab_sub.rename(columns = {'valuenum':'value'})

    rename_df = variable_name.rename(columns = {'abbreviation': 'label'})
    item = pd.concat([using_hsp_itm, rename_df], axis = 0, ignore_index = True)
    
    new_data1 = {'itemid': 70012, 'label': 'blood cultured'}
    new_data2 = {'itemid': 70011, 'label': 'blood cultured'}
    
    item = pd.concat([item, pd.DataFrame([new_data1])], ignore_index=True)
    item = pd.concat([item, pd.DataFrame([new_data2])], ignore_index=True)

    labtest = pd.merge(lab_sub, item, how='left', on='itemid')
    signal = pd.merge(chart_sub, item, how='left', on='itemid')
    microe = pd.merge(mi_sub, item, how='left', on='itemid')
    outpute = pd.merge(oe_sub, item, how='left', on='itemid')
    
    return labtest, signal, microe, outpute

labtest, charte, microee, outputee = item_match(d_labitems, d_items, selected_chartevt, selected_labevt, selected_mi, selected_oe)

In [12]:
# 형태가 같은 테이블 결합
labtest_micro = pd.concat([labtest, microee], axis = 0, ignore_index = True)

chartevent_item_id=[
    220045, 220050, 220051, 220088, 224842, 220277, 228302, 224695, 225668, 227467, 220621,
    227444, 220059, 220060, 223771, 223772, 220074, 220179, 220180, 228640, 220210, 224690,
    224639, 226730, 223830, 220235, 220224, 227686, 227429, 227457, 223761, 223762, 227442,
    227464, 225625, 225612, 220228, 226540, 220545, 225690, 220615, 220739, 223900, 223901, 223835,
    220644, 220587, 220546, 225624

]

labevent_item_id =[
    50817, 50813, 52442, 50809, 50820, 50825, 50971, 52610, 50822, 50893, 52034, 52035, 50863,
    53086, 50983, 52623, 50825, 50811, 51222, 51640, 51221, 51638, 51639, 52028, 50810,
    50885, 53089, 50912, 52546, 52024, 51108, 51109, 51265, 51704, 50886, 70011, 70012, 50861, 50878,
    51300, 51301, 51755, 51006, 52647

]

outputevent_item_id =[
    226557, 226558, 226559, 226560, 226561, 226563, 226564, 226565, 226566, 226567, 226584, 226627, 226631, 226632, 227489
]


condtion_ce = charte.itemid.isin(chartevent_item_id)
condition_le = labtest_micro.itemid.isin(labevent_item_id)
condition_oe = outputee.itemid.isin(outputevent_item_id)

ce = charte[condtion_ce]
le = labtest_micro[condition_le]
oe = outputee[condition_oe]

In [13]:
# 변수명 통일

bulk_changes_pmpe = {
    225792 : 'Invasive_Ventilation', 225794 : 'None_Invasive_Ventilation',
    225402 : 'EKG', 223253 : 'MRI', 225459 : 'CXR', 225975 : 'HeparinSodium',
    225152 : 'HeparinSodium', 225157 : 'Tirofiban', 221261 : 'Abciximab',
    229069 : 'Digoxin', 229617 : 'Epinephrine', 221289 : 'Epinephrine',  225166: 'Potassium_Chloride',
    225876 : 'Imipenem', 225893 : 'Piperacillin_Tazobactam', 226509 : 'extra_blood',
    227080 : 'Fluids_supplement', 220949: 'Dextrose_5%', 225401 : 'blood_cultured', 225437 : 'blood_cultured'
}

bulk_changes_ce = {
    220045 : 'HR', 220050 : 'ABPs', 220051 : 'ABPd',
    220088 : 'CO', 224842 : 'CO', 220277 : 'SpO2',
    228302 : 'RASS', 224695 : 'Peak_insp_P', 227467 : 'INR',
    225668 : 'Lactate', 220621 : 'glucose', 227444 : 'CRP',
    220059 : 'PAPs', 220060 : 'PAPd', 223771 : 'PCWP',
    223772 : 'SvO2', 220074 : 'CVP', 220179 : 'NIBPs', 
    220180 : 'NIBPd', 228640 : 'EtCO2', 220210 : 'RR',
    224690 : 'RR', 224639 : 'weight', 226730 : 'height',
    223830 : 'pH', 220235 : 'pCO2', 220224 : 'PaO2',
    227686 : 'CVOS', 227429 : 'Troponin-T', 227457 : 'Platelet_count',
    223761 : 'temperature_F', 223762 : 'temperature_C', 227442 : 'K+',
    227464 : 'K+', 225625 : 'Ca_non_ionized', 225612 : 'Alkaline_Phosphate',
    220228 : 'hemoglobin', 226540 : 'hematocrit', 220545 : 'hematocrit',
    225690 : 'total_bilirubin', 220615 : 'creatinine', 223835 : 'FiO2',
    220739 : 'Eye_Opening', 223900 : 'verbal_reponse', 223901 : 'mortor_response'
}

bulk_changes_le = {
    50983 : 'sodium', 52623 : 'sodium', 50817 : 'SpO2',
    50813: 'Lactate', 52442 : 'Lactate', 50809 : 'glucose',
    50820 : 'pH', 50825 : 'temperature_C', 50971 : 'K+',
    52610: 'K+', 50822 : 'K+', 50893  : 'Ca_non_ionized', 
    52034 : 'Ca_non_ionized', 52035 : 'Ca_non_ionized', 
    50863 : 'Alkaline_Phosphate', 53086 : 'Alkaline_Phosphate',
    50811 : 'hemoglobin', 51222 : 'hemoglobin', 51640 : 'hemoglobin',
    51221 : 'hematocrit', 51638 : 'hematocrit', 51639 : 'hematocrit', 
    52028 : 'hematocrit', 50810 : 'hematocrit', 
    50912 : 'creatinine', 52546 : 'creatinine', 52024 : 'creatinine', 
    50885 : 'total_bilirubin', 53089 : 'total_bilirubin',
    51108 : 'Urine_Output', 51109 : 'Urine_Output', 50886 : 'blood_cultured',
    70011 : 'blood_cultured', 70012 : 'blood_cultured', 51265 : 'Platelet_count',
    51704 : 'Platelet_count', 50861 : 'ALT', 50878 : 'AST', 51300 : 'WBC', 51301: 'WBC', 51755: 'WBC',
    51006: 'BUN', 52647 :'BUN'
}


# bulk_changes_oe = {
#     226557 : 'Urine_Output', 226558 : 'Urine_Output', 226559 : 'Urine_Output', 226560 : 'Urine_Output', 226561 : 'Urine_Output',
#     226563 : 'Urine_Output', 226564 : 'Urine_Output', 226565 : 'Urine_Output', 226566 : 'Urine_Output', 226567 : 'Urine_Output',
#     226584 : 'Urine_Output', 226627 : 'Urine_Output', 226631 : 'Urine_Output', 226632 : 'Urine_Output', 227488 : 'Urine_Output_in', 227489 : 'Urine_Output_out'
# }

for itemid, new_label in bulk_changes_pmpe.items():
    pmpe.loc[pmpe['itemid'] == itemid, 'label'] = new_label
    
cond = pmpe.itemid.isin([225840, 225875, 225883, 225876, 225893, 229064, 225850, 225855, 225859, 225860, 225865, 225798, 225837, 225838,
    225842, 225843, 225844, 225845, 225847, 225848, 225851, 225857, 225859 ,225862, 225863, 225866, 225868, 225869, 
    225871, 225873, 225875, 225876, 225877 ,225879, 225881, 225882, 225883, 225884, 225885, 225886, 225888, 225889, 
    225890, 225892, 225893, 225895, 225896, 225897, 225898, 225899, 225900, 225902, 225903, 225905, 227691, 228003, 
    229059, 229061, 229064, 229587]) #Antibiotics
idx = pmpe[cond].index

pmpe.loc[idx, 'label'] = 'Antibiotics'

for itemid, new_label in bulk_changes_ce.items():
    ce.loc[ce['itemid'] == itemid, 'label'] = new_label
    
for itemid, new_label in bulk_changes_le.items():
    le.loc[le['itemid'] == itemid, 'label'] = new_label
    
# for itemid, new_label in bulk_changes_oe.items():
#     oe.loc[oe['itemid'] == itemid, 'label'] = new_label

In [14]:
urine = pd.read_csv('../Data/mimic/icu/Urine Output.csv')

In [15]:
urine = urine.rename(columns = {'Urine output':'value'})
urine['label'] = 'Urine_output'

In [16]:
urine_oe = pd.merge(urine, oe[['subject_id', 'hadm_id', 'stay_id']], how = 'left', on = ['subject_id','stay_id'])

In [17]:
oe = urine_oe.copy()

In [18]:
target_stay_id = patient_inf.stay_id.unique()
target_pmpe = pmpe[pmpe.stay_id.isin(target_stay_id)]

pmpe_stay_id = target_pmpe.stay_id.unique()
target_ce = ce[ce.stay_id.isin(pmpe_stay_id)] #47,379,263

ce_subject_id = target_ce.subject_id.unique()
target_le = le[le.subject_id.isin(ce_subject_id)] #14,835,585
target_oe = oe[oe.subject_id.isin(ce_subject_id)] #3,351,069

In [26]:
# lactate_df = target_le[target_le['label']=='Lactate']
# idx = lactate_df[lactate_df.value.isnull()].index
# target_le.drop(idx, inplace = True)
# target_le = target_le.reset_index(drop=True)

In [19]:
# Lab table에 stay id 부여하기 -> charttime을 기준으로 stay_id 매핑

target_ce['charttime'] = pd.to_datetime(target_ce['charttime'])
target_le['charttime'] = pd.to_datetime(target_le['charttime'])

stay_df = stay.dropna()
stay_df['intime'] = pd.to_datetime(stay_df['intime'])
stay_df['outtime'] = pd.to_datetime(stay_df['outtime'])


target_le['stay_id'] = np.nan
result = []

unique_patient_ids = stay['subject_id'].unique()

for p in tqdm(range(len(unique_patient_ids))):
    
    p_id = unique_patient_ids[p]
    
    lab = target_le[target_le['subject_id']==p_id].copy().sort_values('charttime').reset_index(drop=True)
    stay_interest = stay_df[stay_df['subject_id']==p_id].copy()
    
    unique_stay_ids = stay_interest['stay_id'].unique()
    
    for s in  tqdm(range(len(unique_stay_ids)), leave=False):
        
        stay_id = unique_stay_ids[s]
        
        stay_interest2 = stay_interest[stay_interest['stay_id']==stay_id].copy()
        
        indices = np.where((lab['charttime'].values >= stay_interest2['intime'].values) & 
                        (lab['charttime'].values <= stay_interest2['outtime'].values))

        lab['stay_id'].loc[indices[0]] = stay_id

        result.append(lab)
        
result_df = pd.concat(result)

100%|██████████| 50921/50921 [16:46<00:00, 50.59it/s]


In [20]:
# lab event에 stay id가 부여되지 않은 관측치 수(icu stay 테이블에 stay id가 nan인 경우 발생)
print('lab event 관측치 개수: ', len(target_le.subject_id.unique()))
target_le_new = result_df[~(result_df['stay_id'].isnull())]

lab event 관측치 개수:  50701


In [21]:
target_le_new = pd.merge(target_le_new[['subject_id','stay_id', 'charttime', 'itemid', 'value','label']].copy(), stay[['stay_id','hadm_id']], how = 'left', on = ['stay_id'])

In [22]:
target_ce.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value',
       'label'],
      dtype='object')

In [23]:
import time

total = pd.concat([target_ce, target_le_new[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value','label']]], axis = 0, ignore_index = True)
total = total.sort_values(by = ['subject_id', 'charttime'])

print('전체 샘플 수:', len(total))
print('전체 환자 수:', len(total.stay_id.unique()))

전체 샘플 수: 61558668
전체 환자 수: 72772


In [29]:
height = total[total['label']=='height'][['subject_id', 'value']]
weight = total[total['label']=='weight'][['subject_id', 'value']]

In [30]:
dict_h = height.set_index('subject_id')['value'].to_dict()
dict_w = weight.set_index('subject_id')['value'].to_dict()

In [26]:
target_pmpe = pd.merge(target_pmpe, stay[['stay_id','hadm_id']].dropna(), how = 'left', on = ['stay_id'])

In [29]:
total.to_csv('./medical_concept_ICU(MIMIC)/lab_sig_ver3.csv') #checkpoint ce + le + mie
target_pmpe.to_csv('./medical_concept_ICU(MIMIC)/pmpe_ver3.csv') #checkpoint ie, inge
target_oe.to_csv('./medical_concept_ICU(MIMIC)/output_ver3.csv') #checkpoint oe

In [27]:
import pandas as pd
total= pd.read_csv('./medical_concept_ICU(MIMIC)/lab_sig_ver3.csv', index_col = 0) #checkpoint ce + le + mie


In [5]:
target_pmpe = pd.read_csv('./medical_concept_ICU(MIMIC)/pmpe_ver3.csv', index_col = 0) #checkpoint ie, inge
target_oe = pd.read_csv('./medical_concept_ICU(MIMIC)/output_ver3.csv', index_col = 0) #checkpoint oe

In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',  None)

# icu stay
stay = pd.read_csv('../Data/mimic/icu/icustays.csv')

# additional information
inf = pd.read_csv('../Data/mimic/icu/icustays_preprocessed.csv')

patient_inf = demo_inf(inf)

C:\Users\DAHS\AppData\Local\Temp\ipykernel_11628\1964465473.py:8: DtypeWarning: Columns (3,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  stay = pd.read_csv('../Data/mimic/icu/icustays.csv')


In [9]:
import numpy as np
array_1 = np.array(target_pmpe.rename(columns={'amount':'value'})[['subject_id','hadm_id', 'stay_id', 'charttime', 'value', 'label']].copy())
array_2 = np.array(total[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'value', 'label']].copy())
array_3 = np.array(target_oe[['subject_id', 'hadm_id', 'stay_id', 'charttime', 'value', 'label']].copy())

In [10]:
print(array_1.shape)
print(array_2.shape)
print(array_3.shape)

(15926314, 6)
(61558668, 6)
(527322662, 6)


In [11]:
array_12 =np.vstack([array_1, array_2])
array_123 =np.vstack([array_12, array_3])

data = pd.DataFrame(array_123, columns = ['subject_id', 'hadm_id', 'stay_id', 'charttime', 'value', 'label'])

In [12]:
rass_dict = {'Yes':1, 'No':0}
gcs_e_dict = {'Spontaneously':4, 'To Speech':3, 'To Pain':2}
gcs_v_dict = {'Oriented':5, 'No Response-ETT':1, 'Confused':4, 'No Response':1, 'Incomprehensible sounds':2, 'Inappropriate Words':3}
gcs_m_dict = {'Obeys Commands':6, 'Localizes Pain':5, 'No response':1, 'Flex-withdraws':4, 'Abnormal Flexion':3, 'Abnormal extension':2}

rass_idx = data[data['label']=='RASS']['value'].map(rass_dict).index
gcs1_idx = data[data['label']=='Eye_Opening']['value'].map(gcs_e_dict).index
gcs2_idx = data[data['label']=='verbal_reponse']['value'].map(gcs_v_dict).index
gcs3_idx = data[data['label']=='mortor_response']['value'].map(gcs_m_dict).index


data.loc[rass_idx, 'value']  = data[data['label']=='RASS']['value'].map(rass_dict)
data.loc[gcs1_idx, 'value']  = data[data['label']=='Eye_Opening']['value'].map(gcs_e_dict)
data.loc[gcs2_idx, 'value']  = data[data['label']=='verbal_reponse']['value'].map(gcs_v_dict)
data.loc[gcs3_idx, 'value']  = data[data['label']=='mortor_response']['value'].map(gcs_m_dict)

In [13]:
data.to_csv('./medical_concept_ICU(MIMIC)/concat_icu_ver3.csv')

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',  None)

inter_col = ['subject_id','hadm_id','stay_id','charttime','value','label']

chunk_size = 300000
for_pivot = pd.read_csv('./medical_concept_ICU(MIMIC)/concat_icu_ver3.csv', chunksize=chunk_size, usecols = inter_col)

In [2]:
chunks = []
for chunk in for_pivot:
    # 변형 작업 수행
    transformed_chunk = chunk.pivot_table(index = ['charttime','subject_id', 'hadm_id', 'stay_id'], columns = 'label')
    chunks.append(transformed_chunk)

In [3]:
pivot_data = pd.concat(chunks) # 데이터 형태 변환
sample = pivot_data.reset_index()

# 이중 열 해제
sample.columns = [' '.join(col).strip() for col in sample.columns.values]
sample.columns = sample.columns.str.replace('value ', '')

In [276]:
sample['height'] = sample['subject_id'].copy()
sample['weight'] = sample['subject_id'].copy()

sample['weight']  = sample['weight'].map(dict_w)
sample['height']  = sample['height'].map(dict_h)

In [4]:
def Bye_bye(x):
    x.reset_index(drop=True, inplace=True)
    conditions = [
        x['Lactate'].notna().any(),
        x['HR'].notna().any(),
        x['Eye_Opening'].notna().any(),
        x['verbal_reponse'].notna().any(),
        x['mortor_response'].notna().any(),
    ]
    
    if all(conditions):
        return x

truncated_sample = sample.groupby('stay_id').apply(Bye_bye)
truncated_sample.reset_index(drop=True, inplace=True)

In [5]:
print('조건 전 환자 수', len(sample.subject_id.unique()))
print('조건 후 환자 수', len(truncated_sample.subject_id.unique()))

print('조건 전 입원 수', len(sample.stay_id.unique()))
print('조건 후 입원 수', len(truncated_sample.stay_id.unique()))

print('조건 전 전체 관측치 수', len(sample))
print('조건 후 전체 관측치 수', len(truncated_sample))

조건 전 환자 수 50734
조건 후 환자 수 31865
조건 전 입원 수 72775
조건 후 입원 수 41451
조건 전 전체 관측치 수 21081021
조건 후 전체 관측치 수 16903754


In [6]:
# datetime
from datetime import datetime
# Time since icu admission 변수 생성 함수
def TimeAdmission(df, stay):

    sample = df.copy()
    sample['charttime']=pd.to_datetime(sample['charttime'])
    
    make_time = sample[['subject_id', 'stay_id', 'charttime']].copy()
    make_time['intime'] = make_time['stay_id'].copy()
    
    time_icu = stay[['subject_id','stay_id', 'intime']].dropna().copy()
    time_dict = dict(zip(time_icu[['stay_id', 'intime']]['stay_id'], time_icu[['stay_id', 'intime']]['intime']))
    make_time['intime'] = make_time['intime'].map(time_dict)

    make_time['charttime']=pd.to_datetime(make_time['charttime'])
    make_time['intime']=pd.to_datetime(make_time['intime'])


    make_time['Time_since_icu_admission'] = make_time['charttime'] - make_time['intime']
    make_time['Time_since_ICU_admission'] = None
    
    # 죽음의 for 문 X -> apply

    make_time['Time_since_ICU_admission'] = make_time['Time_since_icu_admission'].apply(lambda x: x.total_seconds() / 60)
    
    sample = sample.reset_index(drop=True)
    sample['Time_since_ICU_admission'] =  make_time['Time_since_ICU_admission'].values
    
    return sample

In [7]:
# Time resampling

# 1시간 단위로 리샘플링하고 변수에 대한 평균값(median) 계산
truncated_sample['charttime']=pd.to_datetime(truncated_sample['charttime'])
truncated_sample = truncated_sample.sort_values(by=['stay_id', 'charttime'])

In [8]:
import numpy as np
import warnings

# I expect to see RuntimeWarnings in this block
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    

    total_df_list = []
    unique_stay_ids = truncated_sample['stay_id'].unique()
        
    for i in tqdm(range(len(unique_stay_ids))):
        stay_id = unique_stay_ids[i]
        new_sample_arr = truncated_sample[truncated_sample['stay_id'] == stay_id].reset_index(drop=True).set_index('charttime').groupby(['subject_id', 'hadm_id', 'stay_id']).resample('1H').mean(numeric_only=True)
        
        total_df_list.append(new_sample_arr)
    
resampled_df = pd.concat(total_df_list)

 95%|█████████▌| 39409/41451 [08:26<00:26, 77.83it/s]


KeyboardInterrupt: 

In [ ]:
cut_resampled_df = resampled_df.drop(['subject_id', 'hadm_id', 'stay_id'], axis = 1).reset_index()

In [ ]:
cut_resampled_df.to_csv('check_point_timegrid_resample1010(drop=True).csv')

In [10]:
truncated_sample.to_csv('check_point_noresample1012(drop=True).csv')

In [65]:
check = pd.read_csv('check_point_noresample1012(drop=True).csv', index_col = 0, nrows=1)

In [69]:
len(check.columns[4:])

85

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',  None)

# cut_resampled_df = pd.read_csv('check_point_timegrid_resample1010(drop=True).csv', index_col = 0)
# icu stay
stay = pd.read_csv('../Data/mimic/icu/icustays.csv')

# additional information
inf = pd.read_csv('../Data/mimic/icu/icustays_preprocessed.csv')
patient_inf = demo_inf(inf)

C:\Users\DAHS\AppData\Local\Temp\ipykernel_22604\150392118.py:9: DtypeWarning: Columns (3,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  stay = pd.read_csv('../Data/mimic/icu/icustays.csv')


In [13]:
cut_resampled_df = truncated_sample.copy()

In [14]:
print('총 환자 수 ', len(cut_resampled_df.subject_id.unique()))
print('총 입원 수 ', len(cut_resampled_df.stay_id.unique())) 
print('관측치 수 ', len(cut_resampled_df))

총 환자 수  31865
총 입원 수  41451
관측치 수  16903754


In [15]:
dataset = TimeAdmission(cut_resampled_df, stay)

In [16]:
testing_df = dataset[(dataset['Time_since_ICU_admission'] > 0)].copy() 
# 전체 데이터 관측치 수
print('총 환자 수 ', len(testing_df.subject_id.unique()))
print('총 입원 수 ', len(testing_df.stay_id.unique())) 
print('관측치 수 ', len(testing_df))

총 환자 수  31865
총 입원 수  41451
관측치 수  16845509


In [17]:
#presense of vasoactive/inotropic durg를 확인하기 위해 변수 생성
vaso_ino = ['Norepinephrine', 'Dobutamine', 'Dopamine', 'Epinephrine', 'Milrinone', 'Phenylephrine', 'Phenylephrine (200/250)',
            'Phenylephrine (50/250)', 'Vasopressin', 'Digoxin']

testing_df['vasoactive/inotropic'] = testing_df[vaso_ino].sum(axis = 1)
testing_df['vasoactive/inotropic'] = testing_df['vasoactive/inotropic'].apply(lambda x: 1 if not pd.isna(x) and x != 0 else 0)


# Acute Circulatory Failure
# diagnosis defined by Vincent, Jean-Louis, and Daniel De Backer. "Circulatory shock." New England Journal of Medicine 369.18 (2013): 1726-1734.
def annotation(df):
    data = df.copy()
    
    data['Annotation']=np.nan
    # annotation

    no_circ = data[(data['MAP'] > 65.0) & (data['vasoactive/inotropic'] == 0.0) & (data['Lactate']<=1.5)]
    no_circ['Annotation'] = 'no_circ'
    
    circ = data[((data['MAP'] <= 65.0) | (data['vasoactive/inotropic'] == 1.0)) & (data['Lactate']>1.5)]
    circ['Annotation'] = 'circ'

    result = pd.concat([no_circ, circ], axis = 0)
    result.sort_values(by=['Time_since_ICU_admission', 'subject_id'])
    result.reset_index(drop=True, inplace = True)
    
    return result

In [18]:
### artificial removal 먼저 진행 후 작업해야함

# 각 열의 허용 범위 지정
column_ranges = {
    'HR' : (0, 300),
    'ABPs': (10,300),
    'ABPd': (10,175),
    'NIBPs': (10,300),
    'NIBPd': (10,175),
    'CO'  : (0,20),
    'SpO2': (10,100),
    'INR' :	(0,8),
    'PAPs':	(10,150),
    'PAPd': (10,70),
    'PCWP':	(8,50),
    'CVP': (0,50),
    'Troponin-T': (0, 15000),
    'creatinine': (0, 1500),
    'Platelet_count': (0, 1500),
    'hemoglobin': (0, 200),
    'pH':(7.0, 8.5),
    'Lactate': (0,15),
    'EtCO2': (10, 100),
    'Peak_insp_P': (0,100),
    'RR': (0,45),
    'SvO2': (0,85),
    'hematocrit': (0,70),
    'Potassium_Chloride':(0,100),
    'total_bilirubin':(0,5),
    'sodium':(0,150),
    'temperature_C':(30,45),
    'temperature_F': (40,120),
    'Urine_Output':(0,10000),
    'PaO2': (20 ,500),
    'FiO2': (21, 50),
    'WBC': (0, 30),
    'ALT': (0, 12000) ,
    'AST': (0, 30000) ,
    'BUN': (0, 1000)
}

after_removal = testing_df.copy()

# 범위를 벗어나는 행 필터링
for column, (lower_limit, upper_limit) in column_ranges.items():
    after_removal = after_removal[~((after_removal[column] < lower_limit) | (after_removal[column] > upper_limit))]

    
    print('length of dataset after removing variable {}: {}'.format(column, len(after_removal)))

# 결과 출력
display(after_removal.head())
print('aritificial removal 이후 데이터 관측치 수: ', len(after_removal))
print('제거된 샘플 수: ', len(testing_df) - len(after_removal))
print('총 환자 수 ', len(after_removal.subject_id.unique()))
print('총 입원 수 ', len(after_removal.stay_id.unique())) 
print('관측치 수 ', len(after_removal))

length of dataset after removing variable HR: 16845490
length of dataset after removing variable ABPs: 16845291
length of dataset after removing variable ABPd: 16844729
length of dataset after removing variable NIBPs: 16844580
length of dataset after removing variable NIBPd: 16844084
length of dataset after removing variable CO: 16844077
length of dataset after removing variable SpO2: 16843549
length of dataset after removing variable INR: 16842927
length of dataset after removing variable PAPs: 16842795
length of dataset after removing variable PAPd: 16829510
length of dataset after removing variable PCWP: 16829393
length of dataset after removing variable CVP: 16813466
length of dataset after removing variable Troponin-T: 16813464
length of dataset after removing variable creatinine: 16813449
length of dataset after removing variable Platelet_count: 16813401
length of dataset after removing variable hemoglobin: 16813386
length of dataset after removing variable pH: 16812689
length of

,charttime,subject_id,hadm_id,stay_id,Antibiotics,CXR,Cisatracurium,Dextrose_5%,Digoxin,Diltiazem,Dobutamine,Dopamine,EKG,Epinephrine,Esmolol,Fluids_supplement,HeparinSodium,Invasive_Ventilation,Labetalol,MRI,Metoprolol,Milrinone,Nitroglycerin,Nitroprusside,None_Invasive_Ventilation,Norepinephrine,Phenylephrine,Phenylephrine (200/250),Phenylephrine (50/250),Potassium_Chloride,Rocuronium,Tirofiban,Vasopressin,Vecuronium,Verapamil,Water,blood_cultured,extra_blood,Nesiritide,Aminophylline,Abciximab,ABPd,ABPs,ALT,AST,Alkaline_Phosphate,BUN,CO,CRP,CVOS,CVP,Ca_non_ionized,EtCO2,Eye_Opening,FiO2,HR,INR,K+,Lactate,NIBPd,NIBPs,PAPd,PAPs,PCWP,PaO2,Peak_insp_P,Platelet_count,RASS,RR,SpO2,SvO2,Troponin-T,Urine_Output,WBC,creatinine,glucose,height,hematocrit,hemoglobin,mortor_response,pCO2,pH,sodium,temperature_C,temperature_F,total_bilirubin,verbal_reponse,weight,Urine_output,Time_since_ICU_admission,vasoactive/inotropic
8,2174-09-29 12:10:00,12466550,23998182.0,30000153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0
9,2174-09-29 12:12:00,12466550,23998182.0,30000153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280.0,3.0,0
10,2174-09-29 12:25:00,12466550,23998182.0,30000153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,0
11,2174-09-29 12:27:00,12466550,23998182.0,30000153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,0
12,2174-09-29 12:34:00,12466550,23998182.0,30000153.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.011205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,0


aritificial removal 이후 데이터 관측치 수:  16614378
제거된 샘플 수:  231131
총 환자 수  31865
총 입원 수  41451
관측치 수  16614378


In [19]:
patient_inf

,subject_id,stay_id,intime,age_at_intime,race,gender
0,14046553,35605481,2150-03-03 11:28:38,84.171173,WHITE,M
1,14990224,30503572,2121-08-31 21:29:49,71.665468,WHITE,M
2,17913090,30593599,2145-06-14 16:49:38,50.456715,BLACK,F
3,16058391,37640580,2128-09-18 05:27:41,61.715692,OTHER,M
4,16788749,30610654,2180-01-19 20:40:36,54.051675,WHITE,F
...,...,...,...,...,...,...
76938,14641622,32199257,2135-07-14 13:39:32,59.533066,WHITE,M
76939,10053207,31112026,2199-12-17 23:22:57,59.964313,WHITE,F
76940,13478841,33874605,2188-05-14 16:56:15,77.369057,WHITE,M
76941,10355856,36975675,2157-11-29 15:00:36,69.911303,UNKNOWN,M


In [20]:
demo = patient_inf.copy()
demo_df = pd.get_dummies(demo)
truncated_demo_df = demo_df[['race_ASIAN', 'race_BLACK', 'race_HISPANIC/LATINO','race_OTHER', 'race_UNKNOWN', 'race_WHITE', 'gender_M', 'gender_F']]
truncated_demo_df = truncated_demo_df.astype(int)
truncated_demo_df['stay_id'] = patient_inf['stay_id'].copy()

In [21]:
tdf = after_removal.copy()

asian = truncated_demo_df[['stay_id', 'race_ASIAN']].copy().rename(columns={'stay_id':'stay_id_asian'})
asian_dict = dict(zip(asian['stay_id_asian'], asian['race_ASIAN']))

tdf['stay_id_asian'] = after_removal['stay_id'].copy()
tdf['stay_id_asian'] = tdf['stay_id_asian'].map(asian_dict)

black = truncated_demo_df[['stay_id', 'race_BLACK']].copy().rename(columns={'stay_id':'stay_id_black'})
black_dict = dict(zip(black['stay_id_black'], black['race_BLACK']))

tdf['stay_id_black'] = after_removal['stay_id'].copy()
tdf['stay_id_black'] = tdf['stay_id_black'].map(black_dict)

h_latino = truncated_demo_df[['stay_id', 'race_HISPANIC/LATINO']].copy().rename(columns={'stay_id':'stay_id_hlatino'})
h_latino_dict = dict(zip(h_latino['stay_id_hlatino'], h_latino['race_HISPANIC/LATINO']))

tdf['stay_id_hlatino'] = after_removal['stay_id'].copy()
tdf['stay_id_hlatino'] = tdf['stay_id_hlatino'].map(h_latino_dict)

other = truncated_demo_df[['stay_id', 'race_OTHER']].copy().rename(columns={'stay_id':'stay_id_other'})
other_dict = dict(zip(other['stay_id_other'], other['race_OTHER']))

tdf['stay_id_other'] = after_removal['stay_id'].copy()
tdf['stay_id_other'] = tdf['stay_id_other'].map(other_dict)

unknown = truncated_demo_df[['stay_id', 'race_UNKNOWN']].copy().rename(columns={'stay_id':'stay_id_unkn'})
unknown_dict = dict(zip(unknown['stay_id_unkn'], unknown['race_UNKNOWN']))

tdf['stay_id_unkn'] = after_removal['stay_id'].copy()
tdf['stay_id_unkn'] = tdf['stay_id_unkn'].map(unknown_dict)

white = truncated_demo_df[['stay_id', 'race_WHITE']].copy().rename(columns={'stay_id':'stay_id_white'})
white_dict = dict(zip(white['stay_id_white'], white['race_WHITE']))

tdf['stay_id_white'] = after_removal['stay_id'].copy()
tdf['stay_id_white'] = tdf['stay_id_white'].map(white_dict)

gm = truncated_demo_df[['stay_id', 'gender_M']].copy().rename(columns={'stay_id':'stay_id_gm'})
gm_dict = dict(zip(gm['stay_id_gm'], gm['gender_M']))
tdf['stay_id_gm'] = after_removal['stay_id'].copy()
tdf['stay_id_gm'] = tdf['stay_id_gm'].map(gm_dict)

gw = truncated_demo_df[['stay_id', 'gender_F']].copy().rename(columns={'stay_id':'stay_id_gw'})
gw_dict = dict(zip(gw['stay_id_gw'], gw['gender_F']))

tdf['stay_id_gw'] = after_removal['stay_id'].copy()
tdf['stay_id_gw'] = tdf['stay_id_gw'].map(gw_dict)

age = demo[['stay_id','age_at_intime']].copy().rename(columns = {'stay_id':'stay_id_age'})
age_dict = dict(zip(age['stay_id_age'], age['age_at_intime']))

tdf['stay_id_age'] = after_removal['stay_id'].copy()
tdf['stay_id_age'] = tdf['stay_id_age'].map(age_dict)


tdf = tdf.rename(columns = {'stay_id_asian': 'race_ASIAN', 'stay_id_black': 'race_BLACK', 'stay_id_hlatino': 'race_HISPANIC/LATINO', 'stay_id_other': 'race_OTHER',
                      'stay_id_unkn': 'race_UNKNOWN', 'stay_id_white': 'race_WHITE', 'stay_id_gm':'gender_M', 'stay_id_gw': 'gender_F', 'stay_id_age':'age_at_time'})

In [22]:
monitor_dict ={'NIBPs': 125, 'NIBPd': 75, 'HR': 70, 'RR': 12, 'SpO2': 98,
               'temperature_F': 98, 'temperature_C': 36.5, 'Peak_insp_P':0,
               'FiO2': 21, 'EtCO2': 40, 'ABPs': 125, 'ABPd': 75, 'CVP': 7,
               'PAPs': 22, 'PAPd': 11, 'PCWP': 8, 'SvO2': 70, 'CVOS': 65,
               'ALT':25, 'AST':25, 'WBC': 10, 'BUN': 23, 'CO': 'fill using calcurate',
               'Urine_output': 0}

labtest_dict ={'Alkaline_Phosphate': 85, 'CRP': 4, 'Ca_non_ionized': 8.5,'INR':1,
               'K+': 3.6, 'Platelet_count': 300,'Troponin-T':200,
               'creatinine': 50, 'glucose': 5,'hematocrit': 40, 'hemoglobin':135,
               'pH': 7, 'total_bilirubin': 2, 'sodium': 135, 'PaO2': 87, 'pCO2':40,
               'Eye_Opening':4, 'mortor_response':6, 'verbal_reponse':5, 'RASS':0,
               'Lactate':1.1, 'ALT':25, 'AST':25, 'WBC': 10, 'BUN': 23}

valuehot_col = ['Invasive_Ventilation', 'None_Invasive_Ventilation', 'Water', 'HeparinSodium',
                'Dextrose_5%', 'Dobutamine', 'Dopamine', 'extra_blood', 'Phenylephrine', 'Labetalol',
                'Potassium_Chloride', 'Nitroglycerin', 'Phenylephrine (50/250)', 'Metoprolol',
                'Diltiazem', 'Vasopressin', 'Cisatracurium', 'Epinephrine', 'Milrinone', 'Digoxin',
                'Vecuronium', 'Rocuronium', 'Tirofiban', 'Nitroprusside', 'Esmolol', 'Nesiritide',
                'Aminophylline', 'Abciximab', 'Phenylephrine (200/250)', 'Fluids_supplement',
                'Verapamil', 'Norepinephrine', 'Urine_Output']

one_hot_col = ['CXR', 'EKG', 'Antibiotics', 'blood_cultured', 'MRI', 'vasoactive/inotropic']


physic_col = ['height', 'weight', 'age_at_time']


fill_with_mice = ['race_ASIAN', 'race_BLACK', 'race_HISPANIC/LATINO', 'race_OTHER',
 'race_UNKNOWN', 'race_WHITE', 'gender_F', 'gender_M']


In [81]:
def monitor_imputation(targ, target_sample, mask, monitor_dict):
    
    for col, default_value in monitor_dict.items():
        
        if target_sample[col].isnull().all(): # all value of current variable are Null
            
            targ.loc[mask, col] = targ.loc[mask, col].fillna(default_value)
            
        else:
            first_valid_index = targ.loc[mask, col][targ.loc[mask, col].notnull()].index[0]
            targ.loc[0:first_valid_index, col] = targ.loc[0:first_valid_index, col].fillna(default_value) 
            targ.loc[mask, col] =  targ.loc[mask, col].interpolate(method='linear')
            

def labtest_imputation(targ, target_sample, mask, labtest_dict): 
    
    for col, default_value in labtest_dict.items():
        
        if target_sample[col].isnull().all(): # all value of current variable are Null
            
            if col == 'CO':
                targ.loc[mask, col] = 3.5 * 0.007184 * targ.loc[mask, 'weight'] ** 0.425 * targ.loc[mask, 'height'] ** 0.725
                
            else:
                targ.loc[mask, col] = targ.loc[mask, col].fillna(default_value)
        
        else:
            first_valid_index = targ.loc[mask, col][targ.loc[mask, col].notnull()].index[0]
            first_oberc_index = targ.loc[mask, col].index[0]
            targ.loc[first_oberc_index:first_valid_index, col] = targ.loc[first_oberc_index:first_valid_index, col].fillna(default_value) 
            targ.loc[mask, col] =  targ.loc[mask, col].fillna(method='ffill')
                

def physic_imputation(targ, mask, physic_col):
    
    for py_col in physic_col:

        targ.loc[mask, py_col] = targ.loc[mask, py_col].interpolate(method='linear')
   

def medication_imputation(targ, valuehot_col):
    
    targ[valuehot_col] = targ[valuehot_col].fillna(0)
    
    
def onehot_imputation(targ, one_hot_col):
    
    for bi_col in one_hot_col:
        targ[bi_col] = targ[bi_col].apply(lambda x: 1 if not pd.isna(x) and x != 0 else 0)

In [82]:
def Imputation_process(df, monitor_dict, labtest_dict, valuehot_col, one_hot_col, physic_col):
    global targ, stay_id
    targ = df.copy()
    targ = targ.sort_values(by=['stay_id', 'charttime'])
    medication_imputation(targ, valuehot_col)
    onehot_imputation(targ, one_hot_col)
    
    unique_stay_ids = targ['stay_id'].unique()
    
    for i in tqdm(range(len(unique_stay_ids))):
        stay_id = unique_stay_ids[i]
        mask = targ['stay_id'] == stay_id
        target_sample = targ[mask] 
        physic_imputation(targ, mask, physic_col)
        
        monitor_imputation(targ, target_sample, mask, monitor_dict)
        
        labtest_imputation(targ, target_sample, mask, labtest_dict)
        
    return targ

In [31]:
tdf['height'] = tdf['subject_id'].copy()
tdf['weight'] = tdf['subject_id'].copy()

tdf['weight']  = tdf['weight'].map(dict_w)
tdf['height']  = tdf['height'].map(dict_h)

In [32]:
len(tdf)

16614378

In [33]:
dd = tdf.groupby('stay_id').count()[['charttime']].reset_index()

In [53]:
sbj = dd[dd['charttime'] < 1000].stay_id.unique()
cond = tdf.stay_id.isin(sbj)
after_except = tdf[~cond]

In [54]:
len(after_except)

6534841

In [55]:
len(after_except.stay_id.unique())

3710

In [56]:
len(after_except.subject_id.unique())

3456

In [83]:
import warnings
target_imput = after_except.reset_index(drop=True)
# I expect to see RuntimeWarnings in this block
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    new_result = Imputation_process(target_imput, monitor_dict, labtest_dict, valuehot_col, one_hot_col, physic_col)

100%|██████████| 3710/3710 [52:28<00:00,  1.18it/s]


In [85]:
columns_with_null = new_result.columns[new_result.isnull().any()]
print(columns_with_null)

Index(['CO', 'height', 'weight'], dtype='object')


In [86]:
new_result[['creatinine']].describe()

,creatinine
count,6.534841e+06
mean,2.418807e+00
std,6.728272e+00
min,0.000000e+00
25%,7.000000e-01
50%,1.100000e+00
75%,1.900000e+00
max,1.590000e+02


In [87]:
#creatinine 단위 
# 현재 umol/L로 되어 있음 -> mg/dL
new_result['creatinine(mg/dL)'] = new_result['creatinine']*0.0113

In [88]:
# imputation rule에 적용이 안되는 것들은 0으로 채우고, 표시를해준다.

co_idx = new_result[new_result['CO'].isnull()].index
h_idx = new_result[new_result['height'].isnull()].index
w_idx = new_result[new_result['weight'].isnull()].index

new_result['CO_null_filled'] = 0
new_result.loc[co_idx, 'CO'] = new_result.loc[co_idx, 'CO'].fillna(0)
new_result.loc[co_idx, 'CO_null_filled'] = 1

new_result['weight_null_filled'] = 0
new_result.loc[w_idx, 'weight'] = new_result.loc[w_idx, 'weight'].fillna(0)
new_result.loc[w_idx, 'weight_null_filled'] = 1

new_result['height_null_filled'] = 0
new_result.loc[h_idx, 'height'] = new_result.loc[h_idx, 'height'].fillna(0)
new_result.loc[h_idx, 'height_null_filled'] = 1

In [89]:
new_result['MAP'] = (2*new_result['ABPd']+new_result['ABPs'])/3

In [90]:
print('총 환자 수 ', len(new_result.subject_id.unique()))
print('총 입원 수 ', len(new_result.stay_id.unique())) 
print('관측치 수 ', len(new_result))

총 환자 수  3456
총 입원 수  3710
관측치 수  6534841


In [91]:
new_result.to_csv('지워도됨(imputation-checkpoint1011).csv')

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',  None)


# new_result = pd.read_csv('지워도됨(imputation-checkpoint1011).csv')

In [3]:
# new_result.drop('Unnamed: 0', axis = 1, inplace = True)

In [92]:
after_annoted = annotation(new_result)

In [93]:
after_annoted.Annotation.value_counts()

Annotation
no_circ    3714870
circ        433403
Name: count, dtype: int64

In [94]:
print('총 환자 수 ', len(after_annoted.subject_id.unique()))
print('총 입원 수 ', len(after_annoted.stay_id.unique())) 
print('관측치 수 ', len(after_annoted))

총 환자 수  3456
총 입원 수  3710
관측치 수  4148273


In [95]:
def patient_window(targ, mask):
    
    window_index = targ[mask].index
    targ_sample = targ[mask]
    
    for j, idx in enumerate(window_index):
        current_time = targ.loc[mask, 'Time_since_ICU_admission'].loc[idx]
        endpoint_window = current_time + 480
        
        try:
            window = targ_sample[targ_sample['Time_since_ICU_admission'] < endpoint_window].loc[window_index[j+1]:]
            if (window['Annotation']=='circ').any()==True:
                targ.loc[idx, 'circ_next_8h'] = 1
                
            else:
                targ.loc[idx, 'circ_next_8h'] = 0

        except: # stay 관측치가 1개인 경우
            targ.loc[idx, 'circ_next_8h'] = 0   

def circfailure_labeler(df): 
    targ = df.copy()
    
    targ['circ_next_8h'] = 'None'

    unique_stay_ids = targ['stay_id'].unique()
    for i in tqdm(range(len(unique_stay_ids))):
        stay_id = unique_stay_ids[i]
        mask = targ['stay_id'] == stay_id 
        patient_window(targ, mask)

    return targ     

In [96]:
target = after_annoted.sort_values(by=['stay_id', 'Time_since_ICU_admission']).copy().reset_index(drop=True)

In [97]:
rr = circfailure_labeler(target)

100%|██████████| 3710/3710 [56:53<00:00,  1.09it/s]  


In [98]:
rr.to_csv('check_point_labeling(drop=True).csv')

In [99]:
rr.circ_next_8h.value_counts()

circ_next_8h
0    3634600
1     513673
Name: count, dtype: int64

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',  None)

rr = pd.read_csv('check_point_labeling(drop=True).csv', index_col=0)

In [2]:
rr['weight'] = rr['weight'].astype(float)
rr['height'] = rr['height'].astype(float)

In [3]:
co_idx = rr[rr['CO'] == 'fill using calcurate'].index
rr.loc[co_idx, 'CO'] = 3.5 * 0.007184 * ((rr.loc[co_idx, 'weight'])**0.425)* (rr.loc[co_idx, 'height'])

In [4]:
# additional information
suspected_infection = pd.read_csv('../Data/mimic/hosp/suspected_infection.csv')

# for judgment presence of heart failure 
d_icd = pd.read_csv('../Data/mimic/hosp/d_icd_diagnoses.csv')
diagnosses = pd.read_csv('../Data/mimic/hosp/diagnoses_icd.csv')


# suspected infections

si_icd9_ary = suspected_infection[suspected_infection['icd_version']==9]['icd_code'].values
si_icd9 = np.array(si_icd9_ary[0].split(', '))

si_icd10_ary = suspected_infection[suspected_infection['icd_version']==10]['icd_code'].values
si_icd10 = np.array(si_icd10_ary[0].split(', '))

suspected_infection_icd_code_9_10 =np.hstack([si_icd9, si_icd10])


suspected_infection_df = pd.DataFrame()

for icd in suspected_infection_icd_code_9_10:

    sample = diagnosses[diagnosses['icd_code'].str.startswith(icd)][['subject_id', 'hadm_id']]
    suspected_infection_df = pd.concat([suspected_infection_df, sample])
    
    
heartfailure_before_df = pd.DataFrame()

for icd in ['428', 'I50']:

    sample = diagnosses[diagnosses['icd_code'].str.startswith(icd)][['subject_id', 'hadm_id']]
    heartfailure_before_df = pd.concat([heartfailure_before_df, sample])
    
    

spinal_injury_before_df = pd.DataFrame()

for icd in ['G50', '805', '806']:

    sample = diagnosses[diagnosses['icd_code'].str.startswith(icd)][['subject_id', 'hadm_id']]
    spinal_injury_before_df = pd.concat([spinal_injury_before_df, sample])

In [157]:
suspected_infection_df

,subject_id,hadm_id
73194,10158684,22072764
195927,10427677,23641430
504678,11088311,27128981
852901,11816842,25448380
966550,12051958,25585946
...,...,...
4738409,19960353,20782216
4740805,19965582,22946607
4742849,19970078,22135897
4745901,19975995,26284923


In [5]:
def add_diagnosed_evnet(data, diagnosed_df, make_col):
    diagnosed_df[make_col] = 1
    data[make_col] = 0
    infection_dict = dict(zip(diagnosed_df['hadm_id'], diagnosed_df[make_col]))
    data[make_col] = data['hadm_id'].copy()
    data[make_col] = data[make_col].map(infection_dict).fillna(0)
    
    return data

rr['GCS_score'] = rr['mortor_response'] + rr['verbal_reponse'] + rr['Eye_Opening']

result_1 = add_diagnosed_evnet(rr, suspected_infection_df, 'suspected_infection_diagnosed_stay')
result_2 = add_diagnosed_evnet(result_1, heartfailure_before_df, 'heartfailure_diagnosed_stay')
result_3 = add_diagnosed_evnet(result_2, spinal_injury_before_df, 'spinalcord_diagnosed_stay')

In [8]:
result_3['suspected_infection_diagnosed_stay'].value_counts()

suspected_infection_diagnosed_stay
1.0    3810499
0.0     337774
Name: count, dtype: int64

In [9]:
result_3['heartfailure_diagnosed_stay'].value_counts()

heartfailure_diagnosed_stay
0.0    2828224
1.0    1320049
Name: count, dtype: int64

In [10]:
result_3['spinalcord_diagnosed_stay'].value_counts()

spinalcord_diagnosed_stay
0.0    4013196
1.0     135077
Name: count, dtype: int64

In [6]:
def make_sofa_score(df):
    sample = df.copy()
    
    sample['pao2/fio2'] = sample['PaO2'] / (sample['FiO2']*760/100) # percent to mmHg
    
    sample['Sofa_Respiration'] = None       # Respiratory system
    sample['Sofa_Coagulation'] = None       # Coagulation system
    sample['Sofa_Liver'] = None             # Hepatic system
    sample['Sofa_Cardiovascular'] = None    # Cardiovascular system
    sample['Sofa_GCS'] = None               # Neurologic system
    sample['Sofa_Urine'] = None             # Renal system
    
    
    # Respiratory system
    
    score_0 = sample[sample['pao2/fio2']>=400].index
    score_1 = sample[sample['pao2/fio2']<400].index
    score_2 = sample[sample['pao2/fio2']<300].index
    score_3 = sample[(sample['pao2/fio2']<200)&((sample['None_Invasive_Ventilation']>0)|(sample['Invasive_Ventilation']>0))].index
    score_4 = sample[(sample['pao2/fio2']<100)&((sample['None_Invasive_Ventilation']>0)|(sample['Invasive_Ventilation']>0))].index
    
    sample.loc[score_0, 'Sofa_Respiration'] = 0
    sample.loc[score_1, 'Sofa_Respiration'] = 1
    sample.loc[score_2, 'Sofa_Respiration'] = 2
    sample.loc[score_3, 'Sofa_Respiration'] = 3
    sample.loc[score_4, 'Sofa_Respiration'] = 4
    
    
    # Coagulation system
    
    score_0 = sample[sample['Platelet_count']>=150].index
    score_1 = sample[sample['Platelet_count']<150].index
    score_2 = sample[sample['Platelet_count']<100].index
    score_3 = sample[sample['Platelet_count']<50].index
    score_4 = sample[sample['Platelet_count']<20].index
    
    sample.loc[score_0, 'Sofa_Coagulation'] = 0
    sample.loc[score_1, 'Sofa_Coagulation'] = 1
    sample.loc[score_2, 'Sofa_Coagulation'] = 2
    sample.loc[score_3, 'Sofa_Coagulation'] = 3
    sample.loc[score_4, 'Sofa_Coagulation'] = 4


    # Hepatic system
    
    score_0 = sample[sample['total_bilirubin']<1.2].index
    score_1 = sample[(sample['total_bilirubin']>=1.2)&(sample['total_bilirubin']<=2)].index
    score_2 = sample[(sample['total_bilirubin']>=2)&(sample['total_bilirubin']<=5.9)].index
    score_3 = sample[(sample['total_bilirubin']>=6)&(sample['total_bilirubin']<=11.9)].index
    score_4 = sample[sample['total_bilirubin']>=12].index

    sample.loc[score_0, 'Sofa_Liver'] = 0
    sample.loc[score_1, 'Sofa_Liver'] = 1
    sample.loc[score_2, 'Sofa_Liver'] = 2
    sample.loc[score_3, 'Sofa_Liver'] = 3
    sample.loc[score_4, 'Sofa_Liver'] = 4
    
    
    # Cardiovascular system
    
    score_0 = sample[sample['MAP']>=70].index
    score_1 = sample[sample['MAP']<70].index
    score_2 = sample[(sample['Dopamine']<5)|(sample['Dobutamine']>0)].index
    score_3 = sample[(sample['Dopamine']>5)|(sample['Epinephrine']<=0.1)|(sample['Norepinephrine']<=0.1)].index
    score_4 = sample[(sample['Dopamine']>15)|(sample['Epinephrine']>0.1)|(sample['Norepinephrine']>0.1)].index
    
    sample.loc[score_0, 'Sofa_Cardiovascular'] = 0
    sample.loc[score_1, 'Sofa_Cardiovascular'] = 1
    sample.loc[score_2, 'Sofa_Cardiovascular'] = 2
    sample.loc[score_3, 'Sofa_Cardiovascular'] = 3
    sample.loc[score_4, 'Sofa_Cardiovascular'] = 4
    

    # Neurologic system
    
    score_0 = sample[sample['GCS_score']==15].index
    score_1 = sample[(sample['GCS_score']==13) | (sample['GCS_score']==14)].index
    score_2 = sample[(sample['GCS_score']>=10) | (sample['GCS_score']<=12)].index
    score_3 = sample[(sample['GCS_score']>=6)  | (sample['GCS_score']<=9)].index
    score_4 = sample[sample['GCS_score']<6].index
    
    sample.loc[score_0, 'Sofa_GCS'] = 0
    sample.loc[score_1, 'Sofa_GCS'] = 1
    sample.loc[score_2, 'Sofa_GCS'] = 2
    sample.loc[score_3, 'Sofa_GCS'] = 3
    sample.loc[score_4, 'Sofa_GCS'] = 4 
    
    
    # Renal system
    
    score_0 = sample[sample['creatinine']<1.2].index
    score_1 = sample[(sample['creatinine']>=1.2) | (sample['creatinine']<=1.9)].index
    score_2 = sample[(sample['creatinine']>=2) | (sample['creatinine']<=3.4)].index
    score_3 = sample[(sample['creatinine']>=3.5) | (sample['creatinine']<=4.9) | (sample['Urine_Output']>=200) | (sample['Urine_Output']<=500)].index
    score_4 = sample[(sample['creatinine']>5) | (sample['Urine_Output']<200)].index
    
    sample.loc[score_0, 'Sofa_Urine'] = 0
    sample.loc[score_1, 'Sofa_Urine'] = 1
    sample.loc[score_2, 'Sofa_Urine'] = 2
    sample.loc[score_3, 'Sofa_Urine'] = 3
    sample.loc[score_4, 'Sofa_Urine'] = 4 
    
    
    sample['SoFa_score'] = sample['Sofa_Respiration'] + sample['Sofa_Coagulation'] + sample['Sofa_Liver'] + sample['Sofa_Cardiovascular'] + sample['Sofa_GCS'] + sample['Sofa_Urine']
    
    sample = sample.drop('pao2/fio2', axis = 1)
    
    return sample

new_rr = make_sofa_score(result_3)

In [23]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [8]:
vaso_ino = ['Norepinephrine', 'Dobutamine', 'Dopamine', 'Epinephrine', 'Milrinone', 'Phenylephrine', 'Phenylephrine (200/250)',
            'Phenylephrine (50/250)', 'Vasopressin', 'Digoxin']

# inc_ratio =['NIBPs', 'NIBPd', 'HR', 'RR', 'SpO2', 'temperature_F', 'temperature_C', 'Peak_insp_P',
#             'FiO2', 'EtCO2', 'ABPs', 'ABPd', 'CVP','PAPs', 'PAPd','PCWP','Platelet_count', 'PaO2',
#             'Lactate', 'Eye_Opening', 'mortor_response', 'verbal_reponse','Norepinephrine', 'Dobutamine', 
#             'Dopamine', 'Epinephrine', 'Milrinone', 'Phenylephrine', 'Phenylephrine (200/250)',
#             'Phenylephrine (50/250)', 'Vasopressin', 'Digoxin', 'MAP', 'CO']

# certain_time = [60, 180, 300]

def accumulate(targ, mask, vi_col):

    targ.loc[mask, f'Cumulative_{vi_col}'] = targ.loc[mask, vi_col].cumsum()
    
# def change_amount(targ, mask, rate_col, time_before):
    
#     window_index = targ[mask].index 
#     targ_sample = targ[mask].copy()
    
#     for idx in window_index:
#         current_time = targ.loc[idx, 'Time_since_ICU_admission'] # 180분
#         target_time = current_time - time_before # 180분 - 60분 = 60분
        
#         try:
#             window = targ_sample[(targ_sample['Time_since_ICU_admission']==target_time) | (targ_sample['Time_since_ICU_admission'] > target_time)]
#             target_index = window.index[0] # time before에 해당하는 데이터의 위치
        
#             if target_index in window_index:
#                 targ.loc[idx, f'change_amount_{rate_col}_{time_before}M'] = targ.loc[idx, rate_col] - targ.loc[target_index, rate_col]
#             else:
#                 targ.loc[idx, f'change_amount_{rate_col}_{time_before}M'] = 0
#         except:
#             targ.loc[idx, f'change_amount_{rate_col}_{time_before}M'] = 0
            

def accumulater(df, vaso_ino):
    
    targ = df.copy()
    unique_stay_ids = targ['stay_id'].unique()
    
    for vi_col in vaso_ino:
        
        col_name = f'Cumulative_{vi_col}'
        targ[col_name] = 0
        
    for i in tqdm(range(len(unique_stay_ids))):
        stay_id = unique_stay_ids[i]
        mask = targ['stay_id'] == stay_id
        
        for vi_col in vaso_ino:
            accumulate(targ, mask, vi_col)
    
    return targ

fffin = accumulater(new_rr, vaso_ino)


100%|██████████| 3710/3710 [01:50<00:00, 33.72it/s]


In [9]:
print('전체 샘플의 개수')
print(len(fffin))

print()

print('샘플의 클래스 개수')
print(fffin.circ_next_8h.value_counts())

print()

print('총 Stay 수')
print(len(fffin.stay_id.unique()))

print()

print('총 환자 수')
print(len(fffin.subject_id.unique()))


전체 샘플의 개수
4148273

샘플의 클래스 개수
circ_next_8h
0    3634600
1     513673
Name: count, dtype: int64

총 Stay 수
3710

총 환자 수
3456


In [17]:
fffin.to_csv('medical_concept_ICU(MIMIC)/mimic_ver3.csv')

In [10]:
len(fffin[(fffin['Annotation']=='no_circ')&(fffin['circ_next_8h']==0)])

3624439

In [11]:
#ioc
len(fffin[(fffin['Annotation']=='no_circ')&(fffin['circ_next_8h']==1)])

90431

In [13]:
len(fffin[(fffin['Annotation']=='circ')&(fffin['circ_next_8h']==0)])

10161

In [14]:
len(fffin[(fffin['Annotation']=='circ')&(fffin['circ_next_8h']==1)])

423242

In [52]:
target_col = ['NIBPs', 'NIBPd', 'HR', 'RR', 'SpO2', 'temperature_F', 'temperature_C', 'Peak_insp_P',
            'FiO2', 'EtCO2', 'ABPs', 'ABPd', 'CVP','PAPs', 'PAPd','PCWP','Platelet_count', 'PaO2',
            'Lactate', 'Eye_Opening', 'mortor_response', 'verbal_reponse','Norepinephrine', 'Dobutamine', 
            'Dopamine', 'Epinephrine', 'Milrinone', 'Phenylephrine', 'Phenylephrine (200/250)',
            'Phenylephrine (50/250)', 'Vasopressin', 'Digoxin', 'MAP', 'CO']

diff_60=fffin[['stay_id','NIBPs', 'NIBPd', 'HR', 'RR', 'SpO2', 'temperature_F', 'temperature_C', 'Peak_insp_P',
            'FiO2', 'EtCO2', 'ABPs', 'ABPd', 'CVP','PAPs', 'PAPd','PCWP','Platelet_count', 'PaO2',
            'Lactate', 'Eye_Opening', 'mortor_response', 'verbal_reponse','Norepinephrine', 'Dobutamine', 
            'Dopamine', 'Epinephrine', 'Milrinone', 'Phenylephrine', 'Phenylephrine (200/250)',
            'Phenylephrine (50/250)', 'Vasopressin', 'Digoxin', 'MAP', 'CO']].astype(float).groupby('stay_id').diff(periods=60)

new_column_names = [str(col + '_60') for col in target_col]
diff_60.columns = new_column_names
diff_60 = diff_60.fillna(0)

In [55]:
diff_180=fffin[['stay_id','NIBPs', 'NIBPd', 'HR', 'RR', 'SpO2', 'temperature_F', 'temperature_C', 'Peak_insp_P',
            'FiO2', 'EtCO2', 'ABPs', 'ABPd', 'CVP','PAPs', 'PAPd','PCWP','Platelet_count', 'PaO2',
            'Lactate', 'Eye_Opening', 'mortor_response', 'verbal_reponse','Norepinephrine', 'Dobutamine', 
            'Dopamine', 'Epinephrine', 'Milrinone', 'Phenylephrine', 'Phenylephrine (200/250)',
            'Phenylephrine (50/250)', 'Vasopressin', 'Digoxin', 'MAP', 'CO']].astype(float).groupby('stay_id').diff(periods=180)

new_column_names = [str(col + '_180') for col in target_col]
diff_180.columns = new_column_names
diff_180 = diff_180.fillna(0)

In [56]:
diff_300=fffin[['stay_id','NIBPs', 'NIBPd', 'HR', 'RR', 'SpO2', 'temperature_F', 'temperature_C', 'Peak_insp_P',
            'FiO2', 'EtCO2', 'ABPs', 'ABPd', 'CVP','PAPs', 'PAPd','PCWP','Platelet_count', 'PaO2',
            'Lactate', 'Eye_Opening', 'mortor_response', 'verbal_reponse','Norepinephrine', 'Dobutamine', 
            'Dopamine', 'Epinephrine', 'Milrinone', 'Phenylephrine', 'Phenylephrine (200/250)',
            'Phenylephrine (50/250)', 'Vasopressin', 'Digoxin', 'MAP', 'CO']].astype(float).groupby('stay_id').diff(periods=300)

new_column_names = [str(col + '_300') for col in target_col]
diff_300.columns = new_column_names
diff_300 = diff_300.fillna(0)

In [57]:
rere = pd.concat([diff_60, diff_180], axis = 1)
rerere = pd.concat([rere, diff_300], axis = 1)

In [59]:
maybe_final = pd.concat([fffin, rerere], axis = 1)

In [60]:
maybe_final[maybe_final['stay_id'] == maybe_final.stay_id.unique()[100]][['HR', 'HR_60', 'HR_180']]

,HR,HR_60,HR_180
117416,117.000000,0.0,0.000000
117417,102.500000,0.0,0.000000
117418,95.250000,0.0,0.000000
117419,88.000000,0.0,0.000000
117420,82.333333,0.0,0.000000
...,...,...,...
118751,125.000000,43.0,49.000000
118752,113.666667,31.0,32.666667
118753,102.333333,19.0,16.333333
118754,91.000000,7.0,6.000000


In [62]:
maybe_final.to_csv('medical_concept_ICU(MIMIC)/mimic_ver3.csv')